In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
batch_size = 8
# Data directory
train_directory = 'DB/train'
test_directory = 'DB/test'

# Loading train set + Applying data augmentation on train set
train_datagen = ImageDataGenerator(rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        brightness_range=[0.8,1.0],
        fill_mode="nearest")

# Loading test set
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Generating train batches
train_batches = train_datagen.flow_from_directory(
        train_directory,
        target_size=(224,224),
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle=True)

# Generating test batches
test_batches = test_datagen.flow_from_directory(
        test_directory,
        target_size=(224,224),
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle=True)

In [ ]:
# Model architecture 
# We used pretrained model MobileNetV2 and loaded ImageNet's weight
mobilenet = MobileNetV2(weights = 'imagenet', include_top = True, input_shape=(224,224,3))
mobilenet.layers.pop()
model = Sequential()
model.add(mobilenet)
model.add(Dense(5, activation='softmax', name='predictions'))
model.summary()

In [ ]:
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# Model training
num_train = 4736
num_val = 3568
num_epoch = 10

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['acc'])
model.fit(
        train_batches,
        steps_per_epoch=num_train // batch_size,
        epochs=num_epoch,
        validation_data=test_batches,
        validation_steps=num_val // batch_size,
        callbacks=[model_checkpoint_callback])